In [1]:
import os
os.chdir("../")
%pwd

'd:\\PracticeProjects\\NLP_Insurance_intent_Detection\\nlp_intent'

In [2]:
# This block will be moved to entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [3]:
# Update the configuration manager. This will be moved to config/configurationmanager.py
from intent.constants import *
from intent.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config



In [4]:
# components -> this will be moved to components/init.py
import os
import urllib.request as request
import zipfile
from intent.logging import logger
from intent.utils.common import get_size

class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info:\n {headers}")
        else:
            logger.info(f"{self.config.local_data_file} already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [5]:
# update pipeline. This will be later moved to proper pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-16 01:22:52,160: INFO: common: YAML file: D:\PracticeProjects\NLP_Insurance_intent_Detection\nlp_intent\config\config.yaml loaded successfully]
[2024-05-16 01:22:52,162: INFO: common: YAML file: D:\PracticeProjects\NLP_Insurance_intent_Detection\nlp_intent\params.yaml loaded successfully]
[2024-05-16 01:22:52,163: INFO: common: created directory at: artifacts]
[2024-05-16 01:22:52,164: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-16 01:22:53,031: INFO: 2716360618: artifacts/data_ingestion/data.zip downloaded with following info:
 Connection: close
Content-Length: 217094
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "0bbc574182039047fa9868ef53dc12777fe4e5fb7723a91c3014c4741ffa99b0"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4804:30E548:3888C3:3F4DAF:6